Install the useful packages.

In [1]:
myPackages <- c("easypackages")
#the argument `lib.loc = "~/rlib"` is used because the packages were installed in this folder in my case
lapply(myPackages, library, lib.loc = "~/rlib", character.only = TRUE) 
libraries("dplyr", "tidyr")

[[1]]
[1] "easypackages" "stats"        "graphics"     "grDevices"    "utils"       
[6] "datasets"     "methods"      "base"

Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: tidyr

All packages loaded successfully



Import the GDP PPP per capita (from https://data.worldbank.org/indicator/NY.GDP.PCAP.PP.CD) and democracy (the liberal democracy index from v-dem) data.

For the liberal democracy index,
- v2x_libdem: To what extent is the ideal of liberal democracy achieved
    - Scale: Interval, from low to high (0-1)





In [2]:
GDPraw <- rio::import("API_NY.GDP.PCAP.PP.CD_DS2_en_csv_v2_4250814.csv", skip = 4, header = T)
head(GDPraw)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,⋯,2013,2014,2015,2016,2017,2018,2019,2020,2021,V67
,<chr>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
1,Aruba,ABW,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NA,NA,NA,NA,NA,NA,⋯,36830.157,36782.227,37314.564,37587.066,38893.961,40143.000,39834.224,31182.834,NA,NA
2,Africa Eastern and Southern,AFE,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NA,NA,NA,NA,NA,NA,⋯,3365.684,3503.827,3533.337,3627.474,3702.515,3794.095,3845.619,3687.728,3912.177,NA
3,Afghanistan,AFG,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NA,NA,NA,NA,NA,NA,⋯,2015.515,2069.424,2087.305,1981.118,2058.400,2082.392,2152.190,2078.479,NA,NA
4,Africa Western and Central,AFW,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NA,NA,NA,NA,NA,NA,⋯,4013.197,4208.732,4139.323,4069.006,4115.645,4231.321,4335.409,4241.522,4474.971,NA
5,Angola,AGO,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NA,NA,NA,NA,NA,NA,⋯,7682.476,8179.298,7337.570,7103.226,7310.897,7148.933,6995.300,6478.332,6581.042,NA
6,Albania,ALB,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NA,NA,NA,NA,NA,NA,⋯,10570.964,11259.268,11658.906,12078.843,12770.964,13498.138,13999.424,13632.187,15646.042,NA


In [40]:
LDIraw <- rio::import("V-Dem-CY-Core-v12.csv", 
                      select = c("country_name", "country_text_id", "year",
                                "v2x_libdem", "v2x_libdem_codelow", "v2x_libdem_codehigh", "v2x_libdem_sd"))
head(LDIraw)

,country_name,country_text_id,year,v2x_libdem,v2x_libdem_codelow,v2x_libdem_codehigh,v2x_libdem_sd
,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,Mexico,MEX,1789,0.04,0.027,0.053,0.014
2,Mexico,MEX,1790,0.04,0.027,0.053,0.014
3,Mexico,MEX,1791,0.04,0.027,0.053,0.014
4,Mexico,MEX,1792,0.04,0.027,0.053,0.014
5,Mexico,MEX,1793,0.04,0.027,0.053,0.014
6,Mexico,MEX,1794,0.04,0.027,0.053,0.014


1. Convert GDP data from wide to long format.

2. Drop the rows with `NA` in `GDPperCapPPP`.

3. Filter away the data earlier than 2005 as Gallup data start from 2005.

4. Change the year in GDP data to numeric.

In [55]:
GDPbase <- GDPraw %>% select(- `Indicator Name`, - `Indicator Code`, - V67)
GDPbase <- tidyr::pivot_longer(data = GDPbase, cols = `1960`:`2021`, names_to = "Year", values_to = "GDPperCapPPP")
GDPbase <- GDPbase %>% drop_na %>% filter(Year >= 2005)  %>% as.data.frame
GDPbase <- GDPbase %>% mutate(Year = as.integer(Year))
head(GDPbase)

LDIbase <- LDIraw %>% filter(year >= 2005)

,Country Name,Country Code,Year,GDPperCapPPP
,<chr>,<chr>,<int>,<dbl>
1,Aruba,ABW,2005,34254.14
2,Aruba,ABW,2006,35429.17
3,Aruba,ABW,2007,37366.69
4,Aruba,ABW,2008,38729.56
5,Aruba,ABW,2009,34393.26
6,Aruba,ABW,2010,33784.54


Before merging, we need to check whether the two datasets has the same country names.

If the names are different for the same country, we will use the name in the GDP data.

In [56]:
LDIbase[! LDIbase$`country_name` %in% GDPbase$`Country Name`, "country_name"] %>% unique %>% sort

[1] "Burma/Myanmar"                    "Cape Verde"                      
 [3] "Cuba"                             "Democratic Republic of the Congo"
 [5] "Egypt"                            "Hong Kong"                       
 [7] "Iran"                             "Ivory Coast"                     
 [9] "Kyrgyzstan"                       "Laos"                            
[11] "North Korea"                      "Palestine/Gaza"                  
[13] "Palestine/West Bank"              "Republic of the Congo"           
[15] "Russia"                           "Slovakia"                        
[17] "Somaliland"                       "South Korea"                     
[19] "Syria"                            "Taiwan"                          
[21] "The Gambia"                       "Turkey"                          
[23] "United States of America"         "Venezuela"                       
[25] "Yemen"                            "Zanzibar"

In [57]:
GDPbase[! (GDPbase$`Country Name` %in% LDIbase$`country_name` | grepl(("Africa|Asia|income|World|total|only|Euro|members|Other|ountr"), GDPbase$`Country Name`)), "Country Name"] %>% unique %>% sort

[1] "Antigua and Barbuda"                     
 [2] "Aruba"                                   
 [3] "Bahamas, The"                            
 [4] "Belize"                                  
 [5] "Bermuda"                                 
 [6] "Brunei Darussalam"                       
 [7] "Cabo Verde"                              
 [8] "Caribbean small states"                  
 [9] "Cayman Islands"                          
[10] "Congo, Dem. Rep."                        
[11] "Congo, Rep."                             
[12] "Cote d'Ivoire"                           
[13] "Curacao"                                 
[14] "Dominica"                                
[15] "Early-demographic dividend"              
[16] "Egypt, Arab Rep."                        
[17] "Fragile and conflict affected situations"
[18] "Gambia, The"                             
[19] "Grenada"                                 
[20] "Hong Kong SAR, China"                    
[21] "IDA blend"                               
[22] "Iran, Islamic Rep."                      
[23] "Kiribati"                                
[24] "Korea, Rep."                             
[25] "Kyrgyz Republic"                         
[26] "Lao PDR"                                 
[27] "Late-demographic dividend"               
[28] "Latin America & Caribbean"               
[29] "Macao SAR, China"                        
[30] "Marshall Islands"                        
[31] "Micronesia, Fed. Sts."                   
[32] "Myanmar"                                 
[33] "Nauru"                                   
[34] "North America"                           
[35] "Pacific island small states"             
[36] "Palau"                                   
[37] "Post-demographic dividend"               
[38] "Pre-demographic dividend"                
[39] "Puerto Rico"                             
[40] "Russian Federation"                      
[41] "Samoa"                                   
[42] "San Marino"                              
[43] "Sint Maarten (Dutch part)"               
[44] "Slovak Republic"                         
[45] "Small states"                            
[46] "St. Kitts and Nevis"                     
[47] "St. Lucia"                               
[48] "St. Vincent and the Grenadines"          
[49] "Tonga"                                   
[50] "Turkiye"                                 
[51] "Turks and Caicos Islands"                
[52] "Tuvalu"                                  
[53] "United States"                           
[54] "Venezuela, RB"                           
[55] "West Bank and Gaza"                      
[56] "Yemen, Rep."

In [58]:
LDIbase[LDIbase$country_name %in% "Burma/Myanmar", "country_name"] <- "Myanmar"
LDIbase[LDIbase$country_name %in% "Cape Verde", "country_name"] <- "Cabo Verde"
LDIbase[LDIbase$country_name %in% "Democratic Republic of the Congo", "country_name"] <- "Congo, Dem. Rep."
LDIbase[LDIbase$country_name %in% "Egypt", "country_name"] <- "Egypt, Arab Rep."
LDIbase[LDIbase$country_name %in% "Hong Kong", "country_name"] <- "Hong Kong SAR, China"
LDIbase[LDIbase$country_name %in% "Iran", "country_name"] <- "Iran, Islamic Rep."
LDIbase[LDIbase$country_name %in% "Ivory Coast", "country_name"] <- "Cote d\'Ivoire"
LDIbase[LDIbase$country_name %in% "Kyrgyzstan", "country_name"] <- "Kyrgyz Republic"
LDIbase[LDIbase$country_name %in% "Laos", "country_name"] <- "Lao PDR"
LDIbase[LDIbase$country_name %in% "Palestine/West Bank", "country_name"] <- "West Bank and Gaza"
LDIbase[LDIbase$country_name %in% "Republic of the Congo", "country_name"] <- "Congo, Rep."
LDIbase[LDIbase$country_name %in% "Russia", "country_name"] <- "Russian Federation"
LDIbase[LDIbase$country_name %in% "Slovakia", "country_name"] <- "Slovak Republic"
LDIbase[LDIbase$country_name %in% "South Korea", "country_name"] <- "Korea, Rep."
LDIbase[LDIbase$country_name %in% "The Gambia", "country_name"] <- "Gambia, The"
LDIbase[LDIbase$country_name %in% "Turkey", "country_name"] <- "Turkiye"
LDIbase[LDIbase$country_name %in% "United States of America", "country_name"] <- "United States"
LDIbase[LDIbase$country_name %in% "Venezuela", "country_name"] <- "Venezuela, RB"
LDIbase[LDIbase$country_name %in% "Yemen", "country_name"] <- "Yemen, Rep."


LDIbase[! LDIbase$`country_name` %in% GDPbase$`Country Name`, "country_name"] %>% unique %>% sort
GDPbase[! (GDPbase$`Country Name` %in% LDIbase$`country_name` | grepl(("Africa|Asia|income|World|total|only|Euro|members|Other|ountr"), GDPbase$`Country Name`)), "Country Name"] %>% unique %>% sort

[1] "Cuba"           "North Korea"    "Palestine/Gaza" "Somaliland"    
[5] "Syria"          "Taiwan"         "Zanzibar"

[1] "Antigua and Barbuda"                     
 [2] "Aruba"                                   
 [3] "Bahamas, The"                            
 [4] "Belize"                                  
 [5] "Bermuda"                                 
 [6] "Brunei Darussalam"                       
 [7] "Caribbean small states"                  
 [8] "Cayman Islands"                          
 [9] "Curacao"                                 
[10] "Dominica"                                
[11] "Early-demographic dividend"              
[12] "Fragile and conflict affected situations"
[13] "Grenada"                                 
[14] "IDA blend"                               
[15] "Kiribati"                                
[16] "Late-demographic dividend"               
[17] "Latin America & Caribbean"               
[18] "Macao SAR, China"                        
[19] "Marshall Islands"                        
[20] "Micronesia, Fed. Sts."                   
[21] "Nauru"                                   
[22] "North America"                           
[23] "Pacific island small states"             
[24] "Palau"                                   
[25] "Post-demographic dividend"               
[26] "Pre-demographic dividend"                
[27] "Puerto Rico"                             
[28] "Samoa"                                   
[29] "San Marino"                              
[30] "Sint Maarten (Dutch part)"               
[31] "Small states"                            
[32] "St. Kitts and Nevis"                     
[33] "St. Lucia"                               
[34] "St. Vincent and the Grenadines"          
[35] "Tonga"                                   
[36] "Turks and Caicos Islands"                
[37] "Tuvalu"

The data sets are ready to be merged and exported now.

In [61]:
GDPandLDI <- inner_join(GDPbase, LDIbase %>% select(-country_text_id), by = c("Country Name" = "country_name", "Year" = "year"))
head(GDPandLDI)
dim(GDPandLDI)
write.csv(GDPandLDI, "GDPandLDI.csv", row.names = FALSE)

,Country Name,Country Code,Year,GDPperCapPPP,v2x_libdem,v2x_libdem_codelow,v2x_libdem_codehigh,v2x_libdem_sd
,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Afghanistan,AFG,2005,1023.550,0.133,0.108,0.152,0.022
2,Afghanistan,AFG,2006,1078.907,0.229,0.193,0.259,0.034
3,Afghanistan,AFG,2007,1230.205,0.231,0.200,0.265,0.034
4,Afghanistan,AFG,2008,1273.788,0.234,0.198,0.265,0.035
5,Afghanistan,AFG,2009,1519.311,0.231,0.194,0.260,0.034
6,Afghanistan,AFG,2010,1710.763,0.227,0.195,0.258,0.033


[1] 2857    8

In the future, if we have to merge the LDI and the GDP data sets, we can just merge them using the `country_text_id` and `Country Code` without changing the country names in the LDI data.

The data frame from merging with `country_text_id` (shown in the following) has the same number of rows as the above merged data frame. This is actually indicating that, though how LDI coded the `country_text_id` is unclear, the way they coded the country code and country_text_id is the same.

In [54]:
GDPbase2 <- GDPraw %>% select(-`Indicator Name`, -`Indicator Code`, -V67)
GDPbase2 <- tidyr::pivot_longer(data = GDPbase2, cols = `1960`:`2021`, names_to = "Year", values_to = "GDPperCapPPP")
GDPbase2 <- GDPbase2 %>% drop_na %>% filter(Year >= 2005)  %>% as.data.frame
GDPbase2 <- GDPbase2 %>% mutate(Year = as.integer(Year))

LDIbase2 <- LDIraw %>% filter(year >= 2005)
GDPandLDI2 <- inner_join(GDPbase2, LDIbase2, by = c("Country Code" = "country_text_id", "Year" = "year"))
head(GDPandLDI2)
dim(GDPandLDI2)

,Country Name,Country Code,Year,GDPperCapPPP,country_name,v2x_libdem,v2x_libdem_codelow,v2x_libdem_codehigh,v2x_libdem_sd
,<chr>,<chr>,<int>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,Afghanistan,AFG,2005,1023.550,Afghanistan,0.133,0.108,0.152,0.022
2,Afghanistan,AFG,2006,1078.907,Afghanistan,0.229,0.193,0.259,0.034
3,Afghanistan,AFG,2007,1230.205,Afghanistan,0.231,0.200,0.265,0.034
4,Afghanistan,AFG,2008,1273.788,Afghanistan,0.234,0.198,0.265,0.035
5,Afghanistan,AFG,2009,1519.311,Afghanistan,0.231,0.194,0.260,0.034
6,Afghanistan,AFG,2010,1710.763,Afghanistan,0.227,0.195,0.258,0.033


[1] 2857    9